In [1]:
import numpy as np
import pandas as pd

# to split the datasets
from sklearn.model_selection import train_test_split

# for encoding using feature-engine
from feature_engine.categorical_encoders import RareLabelCategoricalEncoder

In [2]:
data = pd.read_csv('creditApprovalUCI.csv')

data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,1


In [3]:
# make a list with the categorical variables

vars_categorical = ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [4]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['A16'], axis=1),  # predictors
    data['A16'],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((483, 15), (207, 15))

## Count encoding with pandas

In [5]:
# let's inspect the frequency of labels in A7

X_train['A7'].value_counts() / len(X_train)

v          0.573499
h          0.209110
ff         0.084886
bb         0.080745
z          0.014493
dd         0.010352
j          0.010352
Missing    0.008282
n          0.006211
o          0.002070
Name: A7, dtype: float64

In [6]:
def find_frequent_labels(df, variable, tolerance):
    
    temp = df[variable].value_counts()  / len(df)
    
    non_rare = [x for x in temp.loc[temp>tolerance].index.values]
    
    return non_rare

In [7]:
# frequent categories in A7

frequent_cat = find_frequent_labels(X_train, 'A7', 0.05)

frequent_cat

['v', 'h', 'ff', 'bb']

In [8]:
# rare categories in A7

[x for x in X_train['A7'].unique() if x not in frequent_cat]

['dd', 'z', 'j', 'Missing', 'n', 'o']

In [9]:
# let's group rare labels into a new category called rare

X_train['A7'] = np.where(X_train['A7'].isin(frequent_cat), X_train['A7'], 'Rare')
    
X_test['A7'] = np.where(X_test['A7'].isin(frequent_cat), X_test['A7'], 'Rare')

In [10]:
# let's inspect the results

X_train['A7'].value_counts() / len(X_train)

v       0.573499
h       0.209110
ff      0.084886
bb      0.080745
Rare    0.051760
Name: A7, dtype: float64

## One hot encoding with Feature-Engine

In [11]:
# let's separate into train and test sets (again)

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['A16'], axis=1),  # predictors
    data['A16'],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

In [12]:
# Rare category encoder

rare_encoder = RareLabelCategoricalEncoder(tol=0.05, n_categories=4)  

In [13]:
# let's fit the encoder to the train set

rare_encoder.fit(X_train)

RareLabelCategoricalEncoder(n_categories=4, tol=0.05,
                            variables=['A1', 'A4', 'A5', 'A6', 'A7', 'A9',
                                       'A10', 'A12', 'A13'])

In [14]:
rare_encoder.variables

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [15]:
# in the encoder dict we can observe the number of 
# observations per category for each variable

rare_encoder.encoder_dict_

{'A1': array(['a', 'b', 'Missing'], dtype=object),
 'A4': array(['u', 'y', 'Missing', 'l'], dtype=object),
 'A5': array(['g', 'p', 'Missing', 'gg'], dtype=object),
 'A6': Index(['c', 'q', 'w', 'i', 'k', 'ff', 'aa', 'cc', 'm'], dtype='object'),
 'A7': Index(['v', 'h', 'ff', 'bb'], dtype='object'),
 'A9': array(['t', 'f'], dtype=object),
 'A10': array(['t', 'f'], dtype=object),
 'A12': array(['t', 'f'], dtype=object),
 'A13': array(['g', 's', 'p'], dtype=object)}

In [16]:
# let's transform the train and test sets

X_train_enc = rare_encoder.transform(X_train)
X_test_enc = rare_encoder.transform(X_test)

C:\Users\Sole\Anaconda3\envs\feml\lib\site-packages\sklearn\utils\validation.py:933: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  "argument is ignored.", FutureWarning)


In [17]:
X_train_enc.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
596,a,46.08,3.000,u,g,c,v,2.375,t,t,8,t,g,396.0,4159
303,a,15.92,2.875,u,g,q,v,0.085,f,f,0,f,g,120.0,0
204,b,36.33,2.125,y,p,w,v,0.085,t,t,1,f,g,50.0,1187
351,b,22.17,0.585,y,p,ff,ff,0.000,f,f,0,f,g,100.0,0
118,b,57.83,7.040,u,g,m,v,14.000,t,t,6,t,g,360.0,1332


In [18]:
X_test_enc.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
14,a,45.83,10.50,u,g,q,v,5.000,t,t,7,t,g,0.0,0
586,b,64.08,20.00,u,g,Rare,h,17.500,t,t,9,t,g,0.0,1000
140,a,31.25,3.75,u,g,cc,h,0.625,t,t,9,t,g,181.0,0
492,b,39.25,9.50,u,g,m,v,6.500,t,t,14,f,g,240.0,4607
350,a,26.17,2.00,u,g,Rare,Rare,0.000,f,f,0,t,g,276.0,1
